In [1]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 313 kB/s 


# Punto 4

In [45]:
# Punto 4
from rdflib import Graph, Literal, BNode, URIRef, Namespace
# Creamos un grafo vacio
g = Graph()


In [46]:
# Parse a la ontologia
g.parse("Computador.owl",format ="ttl")

<Graph identifier=N352645eab8e74eee90124ea804c43016 (<class 'rdflib.graph.Graph'>)>

In [47]:
# Hora de crear las consultas
# Primera consulta
# Lo que hace esta consulta es recoger todos los nombres de los individuos
consulta1 = """
PREFIX computador: <http://www.computador.org>
SELECT ?nombre
WHERE{
  ?individuo rdf:type owl:NamedIndividual.
  ?individuo rdfs:label ?nombre.
}
"""
for row in g.query(consulta1):
  print(row)

(rdflib.term.Literal('Asus_TUF_Gaming', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('ChromeBook_113180', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Computador_Gamer', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Computador_Hp_250', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Lenovo_IdeaCenter', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Lenovo_Ideapad_3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Macintosh_128K', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Victus_dlh007', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Victus_dlh008', datatype=rdflib.term.URIRef('http://www.w3.or

In [48]:
# Segunda consulta
# Se va a recoger los nombres de las clases cuya clase padre sea Parte_Computador
consulta2 = """
PREFIX computador: <http://www.computador.org>
SELECT ?nombre
WHERE{
  ?objeto rdf:type owl:Class;
    rdfs:label ?nombre;
    rdfs:subClassOf ?propiedad.
  ?propiedad rdfs:label ?nombrePropiedad.
  FILTER(?nombrePropiedad = "Parte_Computador").
}
"""
for row in g.query(consulta2):
  print(row)

(rdflib.term.Literal('Board', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('CPU', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Cooler', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Disco_Duro', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('GPU', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)


# Punto 3

In [49]:
# Importamos la libreria SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST,JSON

# Indicamos el endpoint donde haremos la consulta
sparql = SPARQLWrapper("https://virtuoso-prod-practica-2-sriw-u6r5p2.mo5.mogenius.io/sparql-auth/")


# Damos las credenciales para hacer la consulta
sparql.setHTTPAuth(DIGEST)
sparql.setCredentials("dba", "dba")
sparql.setMethod(POST)

In [57]:
# Consulta e insertar con dbpedia
consulta = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/datatype/>
PREFIX computador: <http://www.computador.org/#>
INSERT {GRAPH <http://www.computador.org/> {?Marca rdf:type owl:NamedIndividual. ?Marca rdfs:label ?Nombre.}
}
where {
  SERVICE <https://dbpedia.org/sparql/> {
    ?Marca dbo:product dbr:Hard_disk_drive .
    ?Marca rdfs:label ?Nombre .
    FILTER(LANG(?Nombre ) = "en")
}
}"""
sparql.setQuery(consulta)
# Obtenemos el resultado y ejecutamos la consulta
results = sparql.query()

# imprimimos el resultado
print(results.response.read())

b'<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/sw/DataAccess/rf1/result2.xsd">\n <head>\n  <variable name="callret-0"/>\n </head>\n <results distinct="false" ordered="true">\n  <result>\n   <binding name="callret-0"><literal>Insert into &lt;http://www.computador.org/&gt;, 36 (or less) triples -- done</literal></binding>\n  </result>\n </results>\n</sparql>\n'


In [55]:
# Consultar e insertar con wikidata
consulta = """
prefix wdt: <http://www.wikidata.org/prop/direct/>
prefix wd: <http://www.wikidata.org/entity/>
PREFIX computador: <http://www.computador.org/#>
INSERT {GRAPH <http://www.computador.org/> {?parteComputador rdfs:subClassOf computador:Parte_Computador.?parteComputador rdfs:label ?parteComputadorLabel.}
}
WHERE {
  SERVICE <https://query.wikidata.org/sparql> {
    ?parteComputador wdt:P361 wd:Q68.
    ?parteComputador rdfs:label ?parteComputadorLabel.
    FILTER( LANG(?parteComputadorLabel) = "en" ).
  }
}

"""
sparql.setQuery(consulta)
# Obtenemos el resultado y ejecutamos la consulta
results = sparql.query()

# imprimimos el resultado
print(results.response.read())

b'<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/sw/DataAccess/rf1/result2.xsd">\n <head>\n  <variable name="callret-0"/>\n </head>\n <results distinct="false" ordered="true">\n  <result>\n   <binding name="callret-0"><literal>Insert into &lt;http://www.computador.org/&gt;, 36 (or less) triples -- done</literal></binding>\n  </result>\n </results>\n</sparql>\n'
